In [ ]:
!pip install torch torchvision pandas numpy matplotlib tqdm

In [ ]:
!pip install torchinfo --quiet

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 35.0 MB/s eta 0:00:00


In [ ]:
!python3 -m pip install --upgrade tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.3 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.9.0.34-py2.py3-none-any.whl size=46629 sha256=4ed0ee2e4e903bd9628f7482ffc5bc253e5c88aacbed7434e505589d47520f38
  Stored in directory: /root/.cache/pip/wheels/d6/d7/3c/0e36cb8283a240ff39eeb4979199490cfeca25700fea039be0
  Created wheel for tensorrt_cu12: filename=tensorrt_cu12-10.9.0.34-py2.py3-none-any.whl size=17465 sha256=3a681b5fe1dc5d75fc76dfccff90ac96f818078707b2e625be26193546430394
  Stored in directory: /root/.cache/pip/wheels/27/0b/b3/cf89d2bb4bb66edec91708b0b785f811b3214d25ac949ba1b7
  Created wheel for tensorrt_cu12_libs: filename=tensorrt_cu12_libs-10.9.0.34-p

In [ ]:
#!/bin/bash
!kaggle datasets download andrewmvd/isic-2019
!unzip -q isic-2019.zip

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/isic-2019
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
100% 9.07G/9.10G [01:03<00:00, 199MB/s]
100% 9.10G/9.10G [01:03<00:00, 154MB/s]


In [ ]:
!pip install pycuda onnx onnxsim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 14.7 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp310-cp310-linux_x86_64.whl size=660661 sha256=9bc3117f15c4465fa2efa0af50db58bd41e303b9bcdd300f1ed38e1ef8aa7c78
  Stored in directory: /root/.cache/pip/wheels/65/53/5f/f5f184c26b7cc503acb77f3456531a6e1fac0ce30c774b9d82
Successfully built pycuda


In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from PIL import Image

In [ ]:
# TensorRT and CUDA
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit  # Initializes CUDA context

import onnx
from onnxsim import simplify

Installing onnxruntime by `/usr/bin/python3 -m pip install onnxruntime`, please wait for a moment..

In [ ]:
# --- Step 2: Organize Data ---
# Load metadata
train_meta = pd.read_csv('ISIC_2019_Training_GroundTruth.csv')
train_meta['image'] = train_meta['image'] + '.jpg'

# Convert one-hot encoding to class labels
classes = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']
train_meta['label'] = train_meta[classes].idxmax(axis=1)
label_to_idx = {cls: idx for idx, cls in enumerate(classes)}


In [ ]:
train_df, temp_df = train_test_split(train_meta, test_size=0.3,
                                    stratify=train_meta['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5,
                                  stratify=temp_df['label'], random_state=42)

# --- Step 3: Define Dataset Class ---
class ISIC2019Dataset(Dataset):
    def __init__(self, df, img_dir='ISIC_2019_Training_Input/ISIC_2019_Training_Input', transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['image'])
        image = Image.open(img_path).convert('RGB')
        label = self.label_to_idx[self.df.iloc[idx]['label']]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# --- Step 4: Data Transforms and Loaders ---
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and loaders
batch_size = 64
train_dataset = ISIC2019Dataset(train_df, transform=train_transform)
val_dataset = ISIC2019Dataset(val_df, transform=val_transform)
test_dataset = ISIC2019Dataset(test_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
# --- Step 6: Model Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Step 5: Handle Class Imbalance ---
train_labels = [label for _, label in train_dataset]
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

In [ ]:
class DINOv2Classifier(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
        self.classifier = nn.Sequential(
            nn.Linear(self.dinov2.embed_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
        for param in self.dinov2.parameters():
            param.requires_grad = False

    def forward(self, x):
        # Get all image features (batch_size, num_patches, embed_dim)
        features = self.dinov2(x)

        # Handle different output formats:
        if features.dim() == 3:  # Standard case with spatial dimensions
            cls_token = features[:, 0, :]  # Extract [CLS] token
        else:  # Fallback for 2D output
            cls_token = features

        return self.classifier(cls_token)

model = DINOv2Classifier().to(device)

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitl14_pretrain.pth
100%|██████████| 1.13G/1.13G [00:06<00:00, 194MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

In [ ]:
num_epochs = 25
best_val_acc = 0.0

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', leave=True)

    for images, labels in train_loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_loop.set_postfix({'loss': f"{loss.item():.4f}"})

    train_loss = train_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    val_loop = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]', leave=True)

    with torch.no_grad():
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            val_loop.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{correct/total:.4f}"
            })

    val_loss /= len(val_loader.dataset)
    val_acc = correct / total
    scheduler.step(val_loss)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_dinov2_isic2019.pth')
        print(f"\nNew best model saved with val acc: {val_acc:.4f}")

    print(f"\nEpoch {epoch+1} Summary:")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

Epoch 1/25 [Val]: 100%|██████████| 60/60 [01:38<00:00,  1.65s/it, loss=1.4488, acc=0.5905]



New best model saved with val acc: 0.5905

Epoch 1 Summary:
Train Loss: 1.6238 | Val Loss: 1.3351 | Val Acc: 0.5905


Epoch 2/25 [Val]: 100%|██████████| 60/60 [01:38<00:00,  1.65s/it, loss=1.3525, acc=0.5805]



Epoch 2 Summary:
Train Loss: 1.3157 | Val Loss: 1.1789 | Val Acc: 0.5805


Epoch 3/25 [Val]: 100%|██████████| 60/60 [01:38<00:00,  1.64s/it, loss=1.2686, acc=0.6547]


...


Epoch 23 Summary:
Train Loss: 0.7576 | Val Loss: 0.8455 | Val Acc: 0.6842


Epoch 24/25 [Val]: 100%|██████████| 60/60 [01:38<00:00,  1.64s/it, loss=1.1368, acc=0.7116]



Epoch 24 Summary:
Train Loss: 0.7356 | Val Loss: 0.8351 | Val Acc: 0.7116


Epoch 25/25 [Val]: 100%|██████████| 60/60 [01:38<00:00,  1.64s/it, loss=0.9677, acc=0.6984]


Epoch 25 Summary:
Train Loss: 0.7290 | Val Loss: 0.8299 | Val Acc: 0.6984


In [ ]:
# --- Step 9: Final Evaluation ---
model.load_state_dict(torch.load('best_dinov2_isic2019.pth'))
model.eval()
test_correct = 0
test_total = 0
test_loop = tqdm(test_loader, desc='Testing', leave=True)

with torch.no_grad():
    for images, labels in test_loop:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        test_loop.set_postfix({'acc': f"{test_correct/test_total:.4f}"})

test_acc = test_correct / test_total
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

<ipython-input-16-f1790a0a500c>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_dinov2_isic2019.pth'))
Testing: 100%|██████████| 60/6


Final Test Accuracy: 0.7166


In [ ]:
import onnxruntime as ort

# Check if ONNX model is valid
try:
    ort.InferenceSession("dinov2_ham10000.onnx")
    print("ONNX model is valid.")
except Exception as e:
    print(f"ONNX validation failed: {e}")

ONNX model is valid.


In [ ]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)  # Suppress verbose logs
builder = trt.Builder(TRT_LOGGER)

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224).cuda()
torch.onnx.export(
    model,
    dummy_input,
    "dinov2_ham10000.onnx",
    opset_version=17,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/patch_embed.py:72: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert H % patch_H == 0, f"Input image height {H} is not a multiple of patch height {patch_H}"
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/patch_embed.py:73: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert W % patch_W == 0, f"Input image width {W} is not a multiple of patch width: {patch_W}"
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/models/vision_transformer.py:183: TracerWarning:

In [ ]:
class HAM10000Calibrator(trt.IInt8EntropyCalibrator2):
    def __init__(self, data_loader, cache_file="ham10000.cache"):
        super().__init__()
        self.data_loader = data_loader
        self.cache_file = cache_file
        self.device_input = cuda.mem_alloc(1 * 3 * 224 * 224 * np.float32().itemsize)
        self.current_idx = 0
        self.data_iter = iter(data_loader)

    def get_batch(self, names):
        try:
            images, _ = next(self.data_iter)
            images = images.numpy().astype(np.float32)
            cuda.memcpy_htod(self.device_input, images.ravel())
            return [int(self.device_input)]
        except StopIteration:
            return None

    def read_calibration_cache(self):
        if os.path.exists(self.cache_file):
            with open(self.cache_file, "rb") as f:
                return f.read()

    def write_calibration_cache(self, cache):
        with open(self.cache_file, "wb") as f:
            f.write(cache)

In [ ]:
TRT_LOGGER = trt.Logger(trt.Logger.ERROR)
with trt.Builder(TRT_LOGGER) as builder, \
     builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network:

    # Parse ONNX
    parser = trt.OnnxParser(network, TRT_LOGGER)
    with open("dinov2_ham10000.onnx", "rb") as f:
        if not parser.parse(f.read()):
            for error in range(parser.num_errors):
                print(parser.get_error(error))
            raise ValueError("Failed to parse ONNX")

    # Configure INT8
    config = builder.create_builder_config()
    config.set_flag(trt.BuilderFlag.INT8)
    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 20 << 30)


    # Set Calibrator
    calib_loader = train_loader
    calibrator = HAM10000Calibrator(calib_loader)
    config.int8_calibrator = calibrator

    config.profiling_verbosity = trt.ProfilingVerbosity.DETAILED
    config.set_flag(trt.BuilderFlag.OBEY_PRECISION_CONSTRAINTS)
    # Build Engine
    engine = builder.build_serialized_network(network, config)

# Save Engine
if engine:
    with open("dinov2_ham10000_int8.engine", "wb") as f:
        f.write(engine)
else:
    print("Engine build failed.")

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


Engine build failed.


<ipython-input-38-9c5fad86d035>:22: DeprecationWarning: Use Deprecated in TensorRT 10.1. Superseded by explicit quantization. instead.
  config.int8_calibrator = calibrator


In [ ]:
# TensorRT inference
start = cuda.Event()
end = cuda.Event()
start.record()
context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
end.record()
cuda.synchronize()
print("Time (ms):", start.time_till(end))

In [ ]:
!sudo apt-get install tensorrt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libnvinfer-bin libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-dispatch10
  libnvinfer-headers-dev libnvinfer-headers-plugin-dev libnvinfer-lean-dev
  libnvinfer-lean10 libnvinfer-plugin-dev libnvinfer-plugin10
  libnvinfer-samples libnvinfer-vc-plugin-dev libnvinfer-vc-plugin10
  libnvinfer10 libnvonnxparsers-dev libnvonnxparsers10 python3-libnvinfer
  python3-libnvinfer-dev python3-libnvinfer-dispatch python3-libnvinfer-lean
The following NEW packages will be installed:
  libnvinfer-bin libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-dispatch10
  libnvinfer-headers-dev libnvinfer-headers-plugin-dev libnvinfer-lean-dev
  libnvinfer-lean10 libnvinfer-plugin-dev libnvinfer-plugin10
  libnvinfer-samples libnvinfer-vc-plugin-dev libnvinfer-vc-plugin10
  libnvinfer10 libnvonnxparsers-dev libnvonnxparsers10 python3-libnvinfer
  python3-l

In [ ]:
!/usr/src/tensorrt/samples/trtexec/

/bin/bash: line 1: /usr/src/tensorrt/samples/trtexec/trtexec: No such file or directory


In [ ]:
!trtexec --onnx=dinov2_ham10000.onnx --int8 --saveEngine=model.engine --verbose

/bin/bash: line 1: trtexec: command not found


In [ ]:
!pip install torchinfo --quiet

In [ ]:
from torchinfo import summary
summary(model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
DINOv2Classifier                              [1, 8]                    --
├─DinoVisionTransformer: 1-1                  [1, 1024]                 1,404,928
│    └─PatchEmbed: 2-1                        [1, 256, 1024]            --
│    │    └─Conv2d: 3-1                       [1, 1024, 16, 16]         (603,136)
│    │    └─Identity: 3-2                     [1, 256, 1024]            --
│    └─ModuleList: 2-2                        --                        --
│    │    └─NestedTensorBlock: 3-3            [1, 257, 1024]            (12,598,272)
│    │    └─NestedTensorBlock: 3-4            [1, 257, 1024]            (12,598,272)
│    │    └─NestedTensorBlock: 3-5            [1, 257, 1024]            (12,598,272)
│    │    └─NestedTensorBlock: 3-6            [1, 257, 1024]            (12,598,272)
│    │    └─NestedTensorBlock: 3-7            [1, 257, 1024]            (12,598,272)
│    │    └─NestedTensorBlock: 

In [ ]:
class DINOv2Full(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
        self.classifier = nn.Sequential(
            nn.Linear(self.dinov2.embed_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
        # for param in self.dinov2.parameters():
        #     param.requires_grad = False

    def forward(self, x):
        # Get all image features (batch_size, num_patches, embed_dim)
        features = self.dinov2(x)

        # Handle different output formats:
        if features.dim() == 3:  # Standard case with spatial dimensions
            cls_token = features[:, 0, :]  # Extract [CLS] token
        else:  # Fallback for 2D output
            cls_token = features

        return self.classifier(cls_token)

model2 = DINOv2Full().to(device)

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


1024


In [ ]:
summary(model2, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
DINOv2Full                                    [1, 8]                    --
├─DinoVisionTransformer: 1-1                  [1, 1024]                 1,404,928
│    └─PatchEmbed: 2-1                        [1, 256, 1024]            --
│    │    └─Conv2d: 3-1                       [1, 1024, 16, 16]         603,136
│    │    └─Identity: 3-2                     [1, 256, 1024]            --
│    └─ModuleList: 2-2                        --                        --
│    │    └─NestedTensorBlock: 3-3            [1, 257, 1024]            12,598,272
│    │    └─NestedTensorBlock: 3-4            [1, 257, 1024]            12,598,272
│    │    └─NestedTensorBlock: 3-5            [1, 257, 1024]            12,598,272
│    │    └─NestedTensorBlock: 3-6            [1, 257, 1024]            12,598,272
│    │    └─NestedTensorBlock: 3-7            [1, 257, 1024]            12,598,272
│    │    └─NestedTensorBlock: 3-8         

In [ ]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import convert_fx, prepare_fx